In [1]:
%load_ext lab_black

In [1]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from imdb import IMDb

In [2]:
titles = pd.read_csv("titles_id.csv", header=None)

In [3]:
list_of_movies = titles.iloc[:, 1].values

In [4]:
ia = IMDb()

In [5]:
title = []
box_office = []
color = []
air_date = []
production_company = []
distribution = []

In [6]:
counter = 0

In [ ]:
# loop to get them all! (or some at least)
for title_id in tqdm(list_of_movies):
    movie = ia.get_movie(title_id[2:])
    title.append(str(movie.get("title") or np.nan))
    box_office.append(str(movie.get("box office") or np.nan))
    color.append(str(movie.get("color") or np.nan))
    air_date.append(str(movie.get("original air date") or np.nan))
    production_company.append(movie.get("production companies") or np.nan)
    distribution.append(movie.get("distributors") or np.nan)
    counter = counter + 1
    tqdm._instances.clear()
    if counter % 100 == 0:
        budget = []
        for element in box_office:
            try:
                budget.append(
                    int(eval(element)["Budget"].split(" ", 1)[0][1:].replace(",", ""))
                )
            except:
                budget.append(np.nan)
        colors = []
        for element in color:
            if "Color" in element:
                colors.append("color")
            elif "Black and White" in element:
                colors.append("black_and_white")
            else:
                colors.append(np.nan)
        company_changed = []
        for element in production_company:
            if element == element:
                company_changed.append([str(element[k]) for k in range(len(element))])
            else:
                company_changed.append(np.nan)
        companies = []
        for element in company_changed:
            if element == element:
                companies.append(",".join(element))
            else:
                companies.append("nothing")
        distribution_changed = []
        for element in distribution:
            if element == element:
                distribution_changed.append(
                    [str(element[k]) for k in range(len(element))]
                )
            else:
                distribution_changed.append(np.nan)
        dist = []
        for element in company_changed:
            if element == element:
                dist.append(",".join(element))
            else:
                dist.append("nothing")
        additional_info = pd.DataFrame(
            list(zip(title, budget, colors, air_date, companies, dist)),
            columns=[
                "title",
                "budget",
                "colors",
                "air_date",
                "production_company",
                "distribution",
            ],
        )
        additional_info.to_csv("data/additional_info.csv")

In [7]:
additional_info = pd.read_csv("data/additional_info.csv")

In [8]:
df_additional = additional_info[
    ["title", "budget", "colors", "air_date", "distribution"]
]

In [9]:
df_additional["air_country"] = df_additional["air_date"].str.extract(r"\((.*?)\)")

C:\Users\Alicja Kocieniewska\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_additional["air_month"] = (
    df_additional["air_date"].str.replace("[^a-zA-Z]", "").str[:3]
)

In [11]:
df_additional.drop(columns="air_date", inplace=True)

In [12]:
def correct_month(row):
    if row["air_month"] in [
        "Jun",
        "Feb",
        "Mar",
        "Apr",
        "May",
        "Jun",
        "Jul",
        "Aug",
        "Sep",
        "Oct",
        "Nov",
        "Dec",
    ]:
        return row["air_month"]
    return np.nan

In [13]:
df_additional["air_month"] = df_additional.apply(lambda row: correct_month(row), axis=1)

In [14]:
df_additional.air_month.unique()

array(['Mar', 'Nov', nan, 'Oct', 'Apr', 'Sep', 'Jun', 'May', 'Feb', 'Aug',
       'Jul', 'Dec'], dtype=object)

In [15]:
dist = df_additional[["title", "distribution"]]

In [16]:
def tidy_split(df, column, sep=",", keep=False):
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return(new_df)

In [17]:
dist = tidy_split(dist, "distribution")

In [18]:
dist_all = dist.distribution.value_counts().keys()[1:200]

In [19]:
def get_dummies(data, column):
    """Return movies dataset with dummy genre variables."""
    subset = data[column].str.get_dummies(sep=",")
    data.index = subset.index
    final_df = pd.concat([data, subset], axis=1)
    final_df.drop(columns=[column], inplace=True)
    return final_df

In [20]:
df_additional["distribution1"], df_additional["distribution2"] = (
    df_additional["distribution"].str.split(",", 1).str
)

In [21]:
df_additional["distribution2"], df_additional["distribution3"] = (
    df_additional["distribution2"].str.split(",", 1).str
)

In [22]:
df_additional["distribution3"], df_additional["distribution4"] = (
    df_additional["distribution3"].str.split(",", 1).str
)

In [23]:
df_additional["distribution4"], df_additional["distribution5"] = (
    df_additional["distribution4"].str.split(",", 1).str
)

In [24]:
df_additional["distribution5"], df_additional["distribution6"] = (
    df_additional["distribution5"].str.split(",", 1).str
)

In [36]:
def distribution_correct(row):
    if row["distribution6"] in dist_all:
        return row["distribution6"]
    return np.nan

In [37]:
df_additional["distribution6"] = df_additional.apply(
    lambda row: distribution_correct(row), axis=1
)

In [38]:
df_additional.iloc[:, 6:] = df_additional.iloc[:, 6:].replace(np.nan, "")

In [39]:
df_additional["distributions"] = (
    df_additional["distribution1"]
    + ","
    + df_additional["distribution2"]
    + ","
    + df_additional["distribution3"]
    + ","
    + df_additional["distribution4"]
    + ","
    + df_additional["distribution5"]
    + ","
    + df_additional["distribution6"]
)

In [40]:
df_additional.drop(
    columns=[
        "distribution1",
        "distribution2",
        "distribution3",
        "distribution4",
        "distribution5",
        "distribution6",
    ],
    inplace=True,
)

In [41]:
df_additional2 = get_dummies(df_additional, "distributions")

In [47]:
df_additional2.drop(columns="distribution", inplace=True)

In [42]:
def decide_color(row):
    if row["colors"] == "color":
        return 0
    if row["colors"] == "black_and_white":
        return 1
    return np.nan

In [43]:
df_additional2["is_black_and_white"] = df_additional2.apply(lambda row: decide_color(row), axis=1)
df_additional2["is_black_and_white"] = df_additional2["is_black_and_white"].astype(bool)
df_additional2.drop(columns="colors", inplace=True)

In [52]:
df_additional2.to_csv("data/additional_info_final.csv")